In [1]:
import cv2
import json
import numpy as np
import random
from copy import deepcopy

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

# set seed

In [2]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

g = torch.Generator()
g.manual_seed(0)

# dataset

In [3]:
class CustomDataset(Dataset):
    def __init__(self, json_path_list):
        self.annotations = []
        for json_path in json_path_list:
            with open(json_path, 'r') as f:
                annotations = json.load(f)
            self.annotations.extend(annotations['annotations'])
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        anno = self.annotations[idx]
        label = anno['label']
        label = np.eye(5)[label].astype(np.float32)
        
        # label smoothing
#         label = label * .9 + 0.1 / 5 
        
        kpts_ = np.array(anno['kpts']).reshape(-1, 2)
        kpts_ = preprocessing(kpts_)
        
        return label, kpts_.astype(np.float32)
    
    
def preprocessing(kpts):
    kpts_ = deepcopy(kpts)
    origin_point = kpts_[0]
    for i in range(21):
        kpts_[i] -= origin_point
    kpts_ /= kpts_.max(axis=0)
    return kpts_.flatten()

In [4]:
ds = CustomDataset([f'./data/annotation_{i}.json' for i in range(5)])
len(ds)

2505

In [5]:
ds[0]

(array([1., 0., 0., 0., 0.], dtype=float32),
 array([0.        , 0.        , 0.73772126, 1.        , 0.8337309 ,
        0.8687269 , 0.9228442 , 0.7656073 , 1.        , 0.70572585,
        0.7634281 , 0.61463314, 0.7689483 , 0.44302133, 0.7661475 ,
        0.33206752, 0.76127577, 0.22502469, 0.6899711 , 0.6205288 ,
        0.65875214, 0.612831  , 0.6724781 , 0.75788313, 0.6950389 ,
        0.75596666, 0.61950314, 0.6486111 , 0.58701164, 0.67132735,
        0.61510015, 0.80165964, 0.6443078 , 0.7869042 , 0.5527041 ,
        0.6827356 , 0.52903885, 0.6963169 , 0.5575288 , 0.7966719 ,
        0.5808849 , 0.81409883], dtype=float32))

In [6]:
total = len(ds)
num_train = int(total * .8)
ds_train, ds_valid = torch.utils.data.random_split(ds, [num_train, total - num_train], generator=g)
print(f'train: {len(ds_train)} / valid: {len(ds_valid)}')

train: 2004 / valid: 501


In [7]:
labels, cnt = np.unique([ds_valid.dataset.annotations[i]['label'] for i in ds_valid.indices], return_counts=True)
print({l:c for l, c in zip(labels, cnt)})

{0: 106, 1: 105, 2: 116, 3: 84, 4: 90}


# model

In [10]:
m = nn.BatchNorm1d(100)
# Without Learnable Parameters
m = nn.BatchNorm1d(100, affine=False)
input = torch.randn(20, 100)
output = m(input)

In [11]:
output.shape

torch.Size([20, 100])

In [8]:
simple_model = nn.Sequential(
    nn.Linear(42, 20),
    nn.BatchNorm1d(20),
    nn.ReLU(True,),
    nn.Linear(20, 10),
    nn.BatchNorm1d(10),
    nn.ReLU(True,),
    nn.Linear(10, 5),
    nn.Sigmoid(),
)

In [13]:
test = simple_model(torch.randn(2, 42))
print('shape:', test.size())

shape: torch.Size([2, 5])


# training

In [14]:
EPOCHS = 60
BS = 64

In [15]:
train_loader = DataLoader(ds_train, BS, shuffle=True, drop_last=True, num_workers=8)
valid_loader = DataLoader(ds_valid, BS, shuffle=False, drop_last=False, num_workers=8)

In [16]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(simple_model.parameters(), lr=2e-3)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=2e-3, epochs=EPOCHS, steps_per_epoch=len(train_loader))

baseline: 0.872255489021956

label smoothing: 0.8343313373253493

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
simple_model = simple_model.to(device)

all_valid_pred = []
all_valid_label = []

for epoch in range(EPOCHS):
    simple_model.train()
    epoch_loss = 0.
    for label, input_ in tqdm(train_loader):
        label, input_ = label.to(device), input_.to(device)
        
        pred = simple_model(input_)
        loss = criterion(pred, label)
        
        epoch_loss += loss.detach().item() * label.size()[0]
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f'[epoch {epoch}] loss: {epoch_loss / len(ds_train)}, lr: {scheduler.get_lr()}')
    
    simple_model.eval()
    
    correct = 0
    for label, input_ in tqdm(valid_loader):
        label, input_ = label.to(device), input_.to(device)
        
        with torch.no_grad():
            pred = simple_model(input_)
            cls_pred = pred.argmax(dim=-1)
            cls_true = label.argmax(dim=-1)
            
            correct += (cls_pred == cls_true).sum().item()
            
            if epoch == EPOCHS - 1:
                all_valid_pred.extend(cls_pred.detach().cpu().numpy().tolist())
                all_valid_label.extend(cls_true.detach().cpu().numpy().tolist())
            
    print(f'--val acc: {correct / len(ds_valid)}')

100%|██████████| 31/31 [00:00<00:00, 53.86it/s]
/usr/lib/python3/dist-packages/torch/optim/lr_scheduler.py:1289: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


[epoch 0] loss: 0.6424797594904186, lr: [9.463683876508033e-05]


100%|██████████| 8/8 [00:00<00:00, 43.03it/s]


--val acc: 0.19161676646706588


100%|██████████| 31/31 [00:00<00:00, 104.43it/s]


...

--val acc: 1.0


100%|██████████| 31/31 [00:00<00:00, 117.08it/s]


[epoch 59] loss: 0.006377458393930675, lr: [1.0911024317473058e-08]


100%|██████████| 8/8 [00:00<00:00, 43.76it/s]

--val acc: 1.0


In [18]:
from sklearn.metrics import confusion_matrix

In [19]:
confusion_matrix(all_valid_label, all_valid_pred)

array([[106,   0,   0,   0,   0],
       [  0, 105,   0,   0,   0],
       [  0,   0, 116,   0,   0],
       [  0,   0,   0,  84,   0],
       [  0,   0,   0,   0,  90]])

### base

acc: 0.872255489021956


```python
array([[ 98,   8,   0,   0,   0],
       [ 14,  57,  34,   0,   0],
       [  0,   8, 108,   0,   0],
       [  0,   0,   0,  84,   0],
       [  0,   0,   0,   0,  90]])
```

### label smoothing / 0.1

acc: 0.8343313373253493

```python
array([[ 97,   8,   1,   0,   0],
       [ 19,  39,  47,   0,   0],
       [  2,   6, 108,   0,   0],
       [  0,   0,   0,  84,   0],
       [  0,   0,   0,   0,  90]])
```

### batchnorm

acc: 1.0

```python
array([[106,   0,   0,   0,   0],
       [  0, 105,   0,   0,   0],
       [  0,   0, 116,   0,   0],
       [  0,   0,   0,  84,   0],
       [  0,   0,   0,   0,  90]])
```

In [21]:
torch.onnx.export(simple_model, input_[:1], "simple_model.onnx", verbose=True, input_names=['input'], output_names=['output'])

graph(%input : Float(1, 42, strides=[42, 1], requires_grad=0, device=cuda:0),
      %0.weight : Float(20, 42, strides=[42, 1], requires_grad=1, device=cuda:0),
      %0.bias : Float(20, strides=[1], requires_grad=1, device=cuda:0),
      %1.weight : Float(20, strides=[1], requires_grad=1, device=cuda:0),
      %1.bias : Float(20, strides=[1], requires_grad=1, device=cuda:0),
      %1.running_mean : Float(20, strides=[1], requires_grad=0, device=cuda:0),
      %1.running_var : Float(20, strides=[1], requires_grad=0, device=cuda:0),
      %3.weight : Float(10, 20, strides=[20, 1], requires_grad=1, device=cuda:0),
      %3.bias : Float(10, strides=[1], requires_grad=1, device=cuda:0),
      %4.weight : Float(10, strides=[1], requires_grad=1, device=cuda:0),
      %4.bias : Float(10, strides=[1], requires_grad=1, device=cuda:0),
      %4.running_mean : Float(10, strides=[1], requires_grad=0, device=cuda:0),
      %4.running_var : Float(10, strides=[1], requires_grad=0, device=cuda:0),
    

In [3]:
import onnxruntime as ort
from tqdm import tqdm
import numpy as np

ort_session = ort.InferenceSession("simple_model.onnx")

for _ in tqdm(range(100)):
    outputs = ort_session.run(
        ['output'],
        {"input": np.random.randn(1, 42).astype(np.float32)},
    )
    

100%|██████████| 100/100 [00:00<00:00, 56043.61it/s]
